## 1. Environment Setup (Linux)

### 1.1. Update Linux Packages

In [6]:
!sudo apt update

0% [Working]

Hit:1 https://dl.yarnpkg.com/debian stable InRelease
Hit:2 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease
Hit:3 https://packages.microsoft.com/repos/microsoft-ubuntu-noble-prod noble InRelease
Hit:4 http://security.ubuntu.com/ubuntu noble-security InRelease         
Hit:5 http://archive.ubuntu.com/ubuntu noble InRelease
Hit:6 http://archive.ubuntu.com/ubuntu noble-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu noble-backports InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
30 packages can be upgraded. Run 'apt list --upgradable' to see them.


### 1.2. Install Python and Packages (pip & venv)

In [7]:
!sudo apt install -y python3 python3-pip python3-venv

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3 is already the newest version (3.12.3-0ubuntu2.1).
python3-pip is already the newest version (24.0+dfsg-1ubuntu1.3).
python3-venv is already the newest version (3.12.3-0ubuntu2.1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


### 1.3. Create a Virtual Environment

In [8]:
!python3 -m venv venv

### 1.4. Activate the Virtual Environment

In [9]:
!source venv/bin/activate

### 1.5. Install the Required Python Packages

In [10]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


## 2. Data Ingestion

### 2.1. Download the Dataset

In [13]:
!wget --user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64)" \
"https://business.yelp.com/external-assets/files/Yelp-JSON.zip" \
-O Yelp-JSON.zip


--2025-12-12 18:17:49--  https://business.yelp.com/external-assets/files/Yelp-JSON.zip
Resolving business.yelp.com (business.yelp.com)... 

52.84.206.14, 52.84.206.127, 52.84.206.74, ...
Connecting to business.yelp.com (business.yelp.com)|52.84.206.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4345335132 (4.0G) [application/zip]
Saving to: ‘Yelp-JSON.zip’

Yelp-JSON.zip       100%[===================>]   4.05G  24.5MB/s    in 2m 56s  

2025-12-12 18:20:46 (23.5 MB/s) - ‘Yelp-JSON.zip’ saved [4345335132/4345335132]



### 2.2. Unzip the downloaded file

In [16]:
!unzip -o Yelp-JSON.zip

Archive:  Yelp-JSON.zip
  inflating: Yelp JSON/Yelp Dataset Documentation & ToS copy.pdf  
  inflating: __MACOSX/Yelp JSON/._Yelp Dataset Documentation & ToS copy.pdf  


  inflating: Yelp JSON/yelp_dataset.tar  

In [ ]:
!rm Yelp-JSON.zip
!rm -rf __MACOSX/

In [15]:
!tar -xvf Yelp\ JSON/yelp_dataset.tar

Dataset_User_Agreement.pdf
yelp_academic_dataset_business.json
yelp_academic_dataset_checkin.json
yelp_academic_dataset_review.json
yelp_academic_dataset_tip.json
yelp_academic_dataset_user.json
^C


In [ ]:
!rm -rf Yelp\ JSON/

## 3. Data Cleaning

## 4. Data Transformation

## 5. Data Querying